In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
sn_1 = pd.read_csv('../data/SNe/Blind15A_07_N2_1214_3860_g.dat', sep='\s+',skiprows=1,\
                  header=None)
sn_1.columns = ['Epoch', 'MJD', 'Mag_auto', 'Mag_err_auto','Flux_auto', 'Flux_err_auto']

In [15]:
sn_1.head()

,Epoch,MJD,Mag_auto,Mag_err_auto,Flux_auto,Flux_err_auto
0,2,57070.115263,20.2165,0.047722,6984.694,231.950368
1,3,57070.200417,20.2684,0.046309,6679.519,219.089241
2,4,57070.269057,20.2668,0.049853,6525.130,221.731573
3,5,57071.044058,20.1971,0.051302,6539.292,195.267704
4,6,57071.111821,20.1903,0.042317,7046.831,185.099072


In [26]:
time_vec = np.array(sn_1['MJD'])*100
time_vec = np.floor(time_vec - time_vec[0])
print time_vec

[    0.     8.    15.    92.    99.   106.   113.   120.   192.   199.
   206.   213.   219.   292.   392.   418.   491.   498.   505.   512.
   519.   697.   703.   996.  1002.  2009.  2503.]
